In [1]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, StackingRegressor
from sklearn.multioutput import MultiOutputRegressor
from random import random
from sklearn.model_selection import cross_validate
import matplotlib
from waveform_methods import binning
import pandas as pd

In [2]:
file = uproot.open('/home/andy/12360.root')
file2 = uproot.open('/home/andy/12362.root')

In [3]:
Xmax1 = file['MCPrimaryInfo']['ghMaxDepth'].array()
valueDepth = file['MCPrimaryInfo']['longNumCharged'].array()
Depth1 = file['MCPrimaryInfo']['longDepth'].array()
chi2_1 = file['CurvatureOnlyParams']['chi2_time'].array()
red1 = file['MCPrimaryInfo']['ghRedChiSqr'].array()
mass1 = [1 for i in range(len(red1))]

good = []
count = 0
for i in range(len(Xmax1)):
    if (red1[i]<300 and chi2_1[i]<5) and Xmax1[i]<900:
        good.append(1)
        count+=1
    else:
        good.append(0)
        
Xmax2 = file2['MCPrimaryInfo']['ghMaxDepth'].array()
valueDepth = file2['MCPrimaryInfo']['longNumCharged'].array()
Depth2 = file2['MCPrimaryInfo']['longDepth'].array()
chi2_2 = file2['CurvatureOnlyParams']['chi2_time'].array()
red2 = file2['MCPrimaryInfo']['ghRedChiSqr'].array()
mass2 = [4 for i in range(len(red2))]

good2 = []
for i in range(len(Xmax2)):
    if (red2[i]<300 and chi2_2[i]<5) and Xmax2[i]<650:
        good2.append(1)
        count+=1
    else:
        good2.append(0)
print(count)

172382


In [4]:
S125_1 = np.log10(file['LaputopParams']['s125'].array())
S125_2 = np.log10(file2['LaputopParams']['s125'].array())

In [5]:
A1 = file['CurvatureOnlyParams']['A'].array()
A2 = file2['CurvatureOnlyParams']['A'].array()
D1 = file['CurvatureOnlyParams']['D'].array()
D2 = file2['CurvatureOnlyParams']['D'].array()
N1 = file['CurvatureOnlyParams']['N'].array()
N2 = file2['CurvatureOnlyParams']['N'].array()
beta1 = file['LaputopParams']['beta'].array()
beta2 = file2['LaputopParams']['beta'].array()
zenith1 = file['Laputop']['zenith'].array()
zenith2 = file2['Laputop']['zenith'].array()
rlogl1 = file['CurvatureOnlyParams']['rlogl'].array()
nmini1 = file['CurvatureOnlyParams']['nmini'].array()
ndf1 = file['CurvatureOnlyParams']['ndf'].array()
llh1 = file['CurvatureOnlyParams']['llh'].array()
rlogl2 = file2['CurvatureOnlyParams']['rlogl'].array()
nmini2 = file2['CurvatureOnlyParams']['nmini'].array()
ndf2 = file2['CurvatureOnlyParams']['ndf'].array()
llh2 = file2['CurvatureOnlyParams']['llh'].array()

Xmax1_new = Xmax1
Xmax2_new = Xmax2

In [6]:
A = np.log10(np.append(A1,A2))
D = np.append(D1,D2)
N = np.append(N1,N2)
S125 = np.append(S125_1,S125_2)
beta = np.log10(np.append(beta1,beta2))
chi2 = np.append(chi2_1,chi2_2)
zenith = np.cos(np.append(zenith1,zenith2))
Xmax = np.append(Xmax1_new,Xmax2_new)
goodness = np.append(good,good2)
red = np.append(red1,red2)
mass = np.append(mass1,mass2)

In [7]:
input_variable = np.array([np.append(np.append(i,j),k) for i,j,k in zip(S125,beta,zenith)])
output = list(zip(Xmax,goodness))

In [8]:
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.2
X_train, X_test, y_train_1, y_test_1 = train_test_split(input_variable,output , test_size=test_size, random_state=seed)

In [11]:
y_train = list(zip(*y_train_1))[1]
y_test = list(zip(*y_test_1))[1]
rng = np.random.RandomState(1)
regr_1 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=7,criterion='friedman_mse'),
                            n_estimators=5, random_state=rng,loss='square')
regr_1.fit(X_train,y_train)
    
y_1= regr_1.predict(X_test)
y_2 = regr_1.predict(X_train)

count_0 = 0
count_02 = 0
for value in y_train:
    if value==0:
        count_0+=1
for value in y_test:
    if value==0:
        count_02+=1
        
count = 0
count2 = 0
for value1,value2 in zip(y_1,y_test):
    if value1<0.5 and value2==0:
        count+=1

for value1,value2 in zip(y_2,y_train):
    if value1<0.5 and value2==0:
        count2+=1

print(count/count_02,count2/count_0,)

0.4552326434757082 0.46006419242972035


In [12]:
check = Xmax>300
check2 = Xmax<900
new_check= []
for i,j in zip(check,check2):
    if i and j:
        new_check.append(True)
    else:
        new_check.append(False)
    
mask = regr_1.predict(input_variable)
mask_convert_pre = []
for i in mask:
    if i>=0.5:
        mask_convert_pre.append(True)
    else:
        mask_convert_pre.append(False)

mask_convert = []
for i,j in zip(new_check,mask_convert_pre):
    if i and j:
        mask_convert.append(True)
    else:
        mask_convert.append(False)

In [13]:
print(np.corrcoef(np.array([A[mask_convert],
                            D[mask_convert],
                            N[mask_convert],
                            S125[mask_convert],
                            beta[mask_convert],
                            zenith[mask_convert],
                            Xmax[mask_convert],
                            chi2[mask_convert],
                            red[mask_convert],
                            mass[mask_convert],
                            goodness[mask_convert]])))

[[ 1.          0.13216003 -0.06992996  0.07621445  0.05489924  0.15042306
   0.09380134  0.15893839 -0.00453584 -0.07362787 -0.17353523]
 [ 0.13216003  1.         -0.02965211 -0.51452004  0.00223243 -0.13862801
  -0.00464256  0.10350347 -0.06472633 -0.15834258 -0.04387069]
 [-0.06992996 -0.02965211  1.          0.05188534 -0.04363735 -0.04930809
  -0.04757658  0.63551711  0.02215767  0.06987297  0.017277  ]
 [ 0.07621445 -0.51452004  0.05188534  1.          0.11606144  0.37025016
   0.28211679 -0.07263374  0.13233394  0.05243804 -0.09357577]
 [ 0.05489924  0.00223243 -0.04363735  0.11606144  1.          0.26102807
   0.22185904 -0.0410097  -0.00821978 -0.19875661  0.05436811]
 [ 0.15042306 -0.13862801 -0.04930809  0.37025016  0.26102807  1.
  -0.05258422 -0.03980583 -0.01332203  0.01897163  0.09013254]
 [ 0.09380134 -0.00464256 -0.04757658  0.28211679  0.22185904 -0.05258422
   1.         -0.01587168  0.05792182 -0.65618207 -0.06744687]
 [ 0.15893839  0.10350347  0.63551711 -0.07263374

In [14]:
mask_new = []
import random
for i,j in zip(mask_convert,mass):
    if (i and j==1) and random.random()<0.15:
        mask_new.append(False)
    else:
        mask_new.append(i)

In [57]:
input_variable2 = np.array([np.append(i,j) for i,j in zip(D[mask_new],beta[mask_new])])
output_new = mass[mask_new]

In [59]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(output_new.reshape(-1,1))

In [60]:
from sklearn.preprocessing import minmax_scale
input_variable_scaled = minmax_scale(input_variable2)

In [61]:
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.2
X_train, X_test, y_train_1, y_test_1 = train_test_split(input_variable2,onehot_encoded, test_size=test_size, random_state=seed)

In [62]:
import keras
from keras import initializers,regularizers
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate, LeakyReLU
from keras.models import Model
import keras.backend as K
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_math_ops as math_ops

In [65]:
input_layer = Input(shape=(2,))

model1 = Dense(7)(input_layer)

model1 = LeakyReLU()(model1)

model2 = Dense(4)(model1)

model2 = LeakyReLU()(model1)

model2 = Concatenate(axis=-1)([model1,model2])

predictions = Dense(2,activation='softmax')(model2)

model = Model(inputs=input_layer,outputs=predictions)

opt = keras.optimizers.RMSprop(decay=1e-5)
#opt= keras.optimizers.Adam(decay=1e-5,lr=3e-4)   

model.compile(optimizer=opt , loss = 'binary_crossentropy')

history = model.fit(X_train,y_train_1,
                    epochs=10,
                    validation_data = (X_test,y_test_1))


Train on 136868 samples, validate on 34218 samples
Epoch 1/10
136868/136868 [==============================] - 4s 27us/step - loss: 0.7167 - val_loss: 0.7831
Epoch 2/10
136868/136868 [==============================] - 3s 25us/step - loss: 0.6824 - val_loss: 0.6997
Epoch 3/10
136868/136868 [==============================] - 3s 26us/step - loss: 0.6791 - val_loss: 0.6718
Epoch 4/10
136868/136868 [==============================] - 3s 25us/step - loss: 0.6757 - val_loss: 0.6864
Epoch 5/10
136868/136868 [==============================] - 3s 25us/step - loss: 0.6725 - val_loss: 0.6657
Epoch 6/10
136868/136868 [==============================] - 4s 27us/step - loss: 0.6697 - val_loss: 0.6651
Epoch 7/10
136868/136868 [==============================] - 4s 27us/step - loss: 0.6686 - val_loss: 0.6591
Epoch 8/10
136868/136868 [==============================] - 4s 26us/step - loss: 0.6672 - val_loss: 0.6655
Epoch 9/10
136868/136868 [==============================] - 4s 26us/step - loss: 0.6662 - val

In [72]:
value = model.predict(X_test)
predicted = np.hstack(onehot_encoder.inverse_transform(value))
true = np.hstack(onehot_encoder.inverse_transform(y_test_1))

In [73]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(true,predicted))

[[12427  4669]
 [ 8077  9045]]


In [74]:
print(12427/(12427+4669))

0.7268951801591016


In [75]:
print(9045/(8077+9045))

0.5282677257329751
